In [ ]:
#r "PoorMansGraphRagQuery/bin/Debug/net8.0/PoorMansGraphRagQuery.dll"
#r "nuget:Azure.AI.OpenAI, 1.0.0-beta.17"
#r "nuget:Azure.Identity"

In [2]:
using PoorMansGraphRagQuery;
var queryEngine = new PoorMansGraphRagQuery.GraphRagQuery();
var userQuery = "What paintings did Leonardo make? Be exhaustive with the content you're given";
var embeddings = await queryEngine.Embed(userQuery);

In [ ]:
//Use embeddings to find Starting Entities in the graph. We use Azure AI Search for this
var graphStartingEntities = await queryEngine.FindStartingEntitiesFromVectorIndex(userQuery, embeddings);
Console.WriteLine(string.Join(Environment.NewLine, graphStartingEntities.Select(x => $"{x.entity}|{x.type}")));


In [ ]:
//Use the LLM to work out what entities and relationships might be important
var entityAndRelationshipTypesToPursue = await queryEngine.GetEntityAndRelationshipTypesToPursue(userQuery);
Console.WriteLine("ENTITIES");
Console.WriteLine(string.Join(Environment.NewLine, entityAndRelationshipTypesToPursue.Entities));
Console.WriteLine();
Console.WriteLine("RELATIONSHIPS");
Console.WriteLine(string.Join(Environment.NewLine, entityAndRelationshipTypesToPursue.Relationships));


In [ ]:
//Traverse the graph - find summaries of interesting entities and relationships. Also attach the chunks of the document most relevant
var graphRagInfo = await queryEngine.TraverseGraph(graphStartingEntities, entityAndRelationshipTypesToPursue.Relationships, entityAndRelationshipTypesToPursue.Entities);
var graphRagChunks = queryEngine.GetChunks(graphRagInfo.chunks);

Console.WriteLine(string.Join(Environment.NewLine, graphRagInfo.graphSummaries));

In [ ]:
//Perform the RAG
var graphRagResult = await queryEngine.GraphRag(graphRagInfo.graphSummaries, userQuery);
Console.WriteLine($"TOKENS: PROMPT {graphRagResult.promptTokens}. COMPLETION:{graphRagResult.completionTokens}");
Console.WriteLine(graphRagResult.response);


In [ ]:
//Perform the RAG with the chunks
var chunks = queryEngine.GetChunks(graphRagInfo.chunks);
var graphRagResult = await queryEngine.GraphRagWithChunks(graphRagInfo.graphSummaries, chunks, userQuery);
Console.WriteLine($"TOKENS: PROMPT {graphRagResult.promptTokens}. COMPLETION:{graphRagResult.completionTokens}");
Console.WriteLine(graphRagResult.response);
